# Time Series Decomposition: Understanding Electricity Load Patterns

In this notebook, we’ll explore how to break down (decompose) a household electricity time series into its fundamental components: **trend**, **seasonality**, and **residuals**. We’ll use several decomposition methods, discuss their strengths and limitations, and learn how to interpret the results. This is a crucial step for anyone learning time series analysis or building forecasting models.

All explanations are written for beginners and are suitable for documenting your learning journey as a blog post.

## 1. Import Required Libraries

We’ll use `polars` for fast data handling, `plotly` for interactive plots, and several time series decomposition tools from `statsmodels` and `utilsforecast`. Some code is for plotting and feature engineering.

In [ ]:
# Import Required Libraries
import polars as pl
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from statsmodels.tsa.seasonal import STL, seasonal_decompose, MSTL
from tsfeatures import tsfeatures, stl_features
from utilsforecast.plotting import plot_series
from utilsforecast.losses import *
from utilsforecast.feature_engineering import fourier, pipeline
from statsmodels.nonparametric.smoothers_lowess import lowess
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from functools import partial
from plotting_utils import (
    plotly_series as plot_series,
    plot_decomposition,
)

pio.templates.default = "plotly_white"

## 2. Load and Prepare the Data

We’ll use half-hourly electricity consumption data for a single household. This keeps things simple and lets us focus on the decomposition methods.

In [ ]:
data = pl.read_parquet(
    "data/london_smart_meters/preprocessed/london_smart_meters_merged_block_0-7.parquet"
)
timestamp = data.group_by("LCLid").agg(
    pl.datetime_range(
        start=pl.col("start_timestamp"),
        end=pl.col("start_timestamp").dt.offset_by(
            pl.format("{}m", pl.col("series_length").sub(1).mul(30))
        ),
        interval="30m",
    ).alias("ds"),
)
data = timestamp.join(data, on="LCLid", how="inner").rename(
    {"LCLid": "unique_id", "energy_consumption": "y"}
)
data = data.filter(pl.col("file").eq("block_7"))
id_ = "unique_id"
time_ = "ds"
target_ = "y"
data = data.select([time_, id_, target_]).explode([time_, target_])
selected_id = "MAC000193"
data = data.filter(pl.col(id_).eq(selected_id))
data.head()

## 3. Visualize the Time Series

Let’s plot the raw time series to get a sense of its structure before decomposition.

In [ ]:
plot_series(data, max_insample_length=1000)

## 4. Traditional Seasonal Decomposition

We start with the classic approach: decompose the series into **trend**, **seasonality**, and **residuals** using moving averages and period averages. This is a great way to build intuition for what decomposition means.

Seasonal decomposition is a fundamental technique in time series analysis that separates a time series into three main components:
1. **Trend**: The long-term progression of the series, representing the underlying direction.
2. **Seasonality**: The repeating patterns or cycles within a fixed period.
3. **Residual**: The remaining variation after removing the trend and seasonality, often referred to as noise or irregularity.

This decomposition can be expressed mathematically as:
- **Additive Model**: $y_t = T_t + S_t + R_t$
- **Multiplicative Model**: $y_t = T_t \times S_t \times R_t$

Here, $y_t$ is the observed value at time $ t $, $ T_t $ is the trend component, $ S_t $ is the seasonal component, and $ R_t $ is the residual component.

#### 1. Extracting the Trend Component
The trend component $ T_t $ is typically estimated using a **moving average**. A moving average smooths the time series by averaging values over a fixed window, helping to remove short-term fluctuations and highlight the long-term trend.

For a centered moving average with a window size $ k $, the trend at time $ t $ is computed as:
$$
T_t = \frac{1}{k} \sum_{i=-\frac{k-1}{2}}^{\frac{k-1}{2}} y_{t+i}
$$
Here, $ k $ is often chosen to match the periodicity of the data (e.g., 12 for monthly data with yearly seasonality).

For example, if the data has a daily seasonality with 48 half-hourly observations per day, a moving average with a window size of 48 can be used to compute the trend.

#### 2. Extracting the Seasonal Component
Once the trend $ T_t $ is estimated, the seasonal component $ S_t $ can be computed by removing the trend from the original series and averaging the detrended values for each period.

For an additive model:
$$
S_t = \frac{1}{n} \sum_{j=1}^{n} (y_{t+jp} - T_{t+jp})
$$
For a multiplicative model:
$$
S_t = \frac{1}{n} \sum_{j=1}^{n} \frac{y_{t+jp}}{T_{t+jp}}
$$
Here, $ p $ is the period of seasonality (e.g., 48 for daily seasonality), and $ n $ is the number of complete cycles in the data.

The seasonal component is computed separately for each time point within the period (e.g., for each half-hour in a day), and the resulting seasonal values are repeated across the entire time series.

#### 3. Computing the Residual Component
Finally, the residual component $ R_t $ is obtained by subtracting the trend and seasonal components from the original series:
- For an additive model: $ R_t = y_t - T_t - S_t $
- For a multiplicative model: $ R_t = \frac{y_t}{T_t \times S_t} $

#### Full Workflow
1. Compute the trend using a moving average.
2. Subtract the trend from the original series to get the detrended series.
3. Compute the seasonal component by averaging the detrended values for each period.
4. Subtract the trend and seasonal components from the original series to get the residual.

In [ ]:
res = seasonal_decompose(
    data.select(pl.col(target_).forward_fill()),
    period=48,  # daily seasonality for half-hourly data
    model="additive",
    extrapolate_trend="freq",
    filt=np.repeat(1 / (30 * 48), 30 * 48),
)
time = data.get_column(time_).to_numpy()

In [ ]:
fig = plot_decomposition(
    time,
    res.observed,
    res.seasonal,
    res.trend,
    res.resid,
)
fig.show()

### Interpretation

- **Flat Trend:** The trend is essentially flat, as expected for a single household.
- **Seasonality:** Strong daily cycles are visible.
- **Residuals:** Still show some structure, suggesting more complex seasonality or patterns remain.

This traditional decomposition method is simple yet powerful for understanding the structure of a time series. However, there are some caveats with this method when it comes to computing trend and seasonality

#### Caveats of Using Moving Average for Trend Extraction

While moving average is simple and intuitive, it has several limitations:

- **Edge Effects**: The moving average cannot be computed at the start and end of the series, leading to missing values at the boundaries.
- **Fixed Window**: It uses a fixed window size, which may not adapt well to changes in the trend or to non-stationary data.
- **Inflexibility**: It cannot capture non-linear or rapidly changing trends, as it only smooths over a fixed interval.
- **Sensitivity to Outliers**: Moving averages can be influenced by outliers within the window, distorting the estimated trend.

#### Caveats of Seasonality Computation in Simple Seasonal Decomposition

- **Assumes Constant Seasonality**: The method assumes the seasonal pattern is fixed and repeats identically in every cycle, which may not be true for real-world data where seasonality can evolve over time.
- **Sensitive to Outliers and Missing Data**: Outliers or missing values in the detrended series can distort the estimated seasonal pattern, as the seasonal component is computed by averaging across periods.
- **Requires Complete Cycles**: Accurate seasonal estimation depends on having a sufficient number of complete seasonal cycles; otherwise, the seasonal averages may be biased.
- **Ignores Multiple or Changing Seasonalities**: Simple decomposition cannot handle multiple overlapping seasonal patterns or seasonality that changes in strength or shape over time. Advanced methods like STL or MSTL are needed for such cases.

In [ ]:
y = data.select(pl.col(target_).forward_fill()).to_numpy().squeeze()
# Compute trend using moving average with window size equal to period (48 for daily seasonality)
window = 48
trend = np.convolve(y, np.ones(window) / window, mode="same")

detrended = y - trend
period = 48
period_averages = np.array([np.nanmean(detrended[i::period]) for i in range(period)])
period_averages -= np.mean(period_averages)

seasonal = np.tile(period_averages, len(detrended) // period + 1)[: len(detrended)]

residual = y - trend - seasonal

In [ ]:
plot_decomposition(
    time=time,
    observed=y,
    trend=trend,
    seasonal=seasonal,
    resid=residual,
)

#### When to Use Seasonally Adjusted Data vs. Extracted Seasonal Components

**Seasonally adjusted data** is obtained by removing the estimated seasonal component from the original time series. This is useful when you want to analyze or model the underlying trend and irregular fluctuations without the influence of recurring seasonal patterns. Typical use cases include:

- **Trend Analysis:** To study long-term changes or structural breaks in the data without seasonal noise.
- **Regression Modeling:** When building models to explain or forecast the underlying process, removing seasonality can improve model accuracy and interpretability.
- **Anomaly Detection:** Outliers and unusual events are easier to detect in seasonally adjusted data, as regular seasonal effects have been removed.
- **Comparing Across Time:** Seasonally adjusted values allow for meaningful comparisons between different periods, unaffected by predictable seasonal swings.

**Extracted seasonal components** are used when the focus is on understanding, visualizing, or modeling the seasonal patterns themselves. Use cases include:

- **Seasonality Analysis:** To quantify and interpret the strength, timing, and nature of recurring cycles (e.g., daily, weekly, yearly).
- **Feature Engineering:** Incorporating seasonal indices as features in machine learning models to capture periodic effects.
- **Forecasting:** When future seasonal effects need to be explicitly modeled or projected, the extracted seasonal component can be added back to trend and residual forecasts.
- **Business Insights:** Understanding when peaks and troughs occur (e.g., high electricity usage times) for operational planning or policy decisions.

**Summary Table:**

| Use Case                        | Use Seasonally Adjusted Data | Use Extracted Seasonal Component |
|----------------------------------|:---------------------------:|:--------------------------------:|
| Trend/Structural Analysis        | ✔️                          |                                  |
| Anomaly/Outlier Detection        | ✔️                          |                                  |
| Regression/ML Modeling           | ✔️                          |                                  |
| Comparing Across Time Periods    | ✔️                          |                                  |
| Understanding Seasonality        |                             | ✔️                               |
| Feature Engineering (Seasonality)|                             | ✔️                               |
| Forecasting with Seasonality     |                             | ✔️ (add to trend/residual)       |
| Business/Operational Planning    |                             | ✔️                               |

In practice, both seasonally adjusted data and extracted seasonal components are valuable—choose based on your analysis goals.

In [ ]:
observed = res.observed
trend = res.trend
seasonal = res.seasonal
seasonally_adjusted = observed - seasonal

fig = go.Figure()
fig.add_trace(
    go.Scatter(x=data.get_column(time_), y=observed, mode="lines", name="Observed")
)
fig.add_trace(
    go.Scatter(
        x=data.get_column(time_),
        y=seasonally_adjusted,
        mode="lines",
        name="Seasonally Adjusted",
    )
)
fig.add_trace(go.Scatter(x=time, y=trend, mode="lines", name="Trend"))
fig.update_layout(
    title="Observed vs Seasonally Adjusted",
    xaxis_title="Time",
    yaxis_title="Value",
    autosize=False,
    width=1200,
    height=600,
    legend=dict(
        font=dict(size=12),
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1,
    ),
)
fig.update_xaxes(type="date", range=["2012-11-01", "2012-12-31"])
fig.show()

## 5. STL: Seasonal-Trend Decomposition using LOESS

STL is a robust, flexible method that can adapt to changes in trend and seasonality. It uses LOESS (locally weighted regression) for smoothing, which is more flexible than moving averages.

Unlike traditional decomposition methods, STL is highly adaptable and can handle complex time series data with varying seasonal patterns and trends.

#### Mathematical Formulation of STL Decomposition

Given a time series $y_t$, STL decomposes it into three components:

$$
y_t = T_t + S_t + R_t
$$

where:
- $T_t$ is the **trend** component,
- $S_t$ is the **seasonal** component,
- $R_t$ is the **residual** (remainder) component.

**Trend Estimation:**  
The trend component $T_t$ is estimated by applying LOESS (locally weighted regression) smoothing to the seasonally adjusted series:

$$
T_t = \text{LOESS}(y_t - S_t)
$$

**Seasonality Estimation:**  
The seasonal component $S_t$ is estimated by applying LOESS smoothing to the detrended series, typically using a window equal to the seasonal period:

$$
S_t = \text{LOESS}(y_t - T_t)
$$

This process is performed iteratively, refining $T_t$ and $S_t$ at each step.

**Residual Calculation:**  
The residual is computed as:

$$
R_t = y_t - T_t - S_t
$$

#### Advantages of LOESS (Locally Estimated Scatterplot Smoothing)

LOESS (or LOWESS) is a non-parametric regression method that fits simple models to localized subsets of the data:

- **Adaptive Smoothing**: LOESS adapts the amount of smoothing based on the local structure of the data, allowing it to capture non-linear and varying trends.
- **Handles Edges Better**: LOESS can provide trend estimates at the boundaries of the series, reducing edge effects.
- **Robustness**: It can be made robust to outliers by using weighted regression.
- **Flexibility**: LOESS does not assume a fixed window or linearity, making it suitable for complex, real-world time series.

In summary, LOESS provides a more flexible and accurate trend estimation, especially when the underlying trend is non-linear or changes over time, whereas moving average is best suited for simple, stationary trends.

For more information about LOESS, see:
- [Lowess and Loess, Clearly Explained!!! (YouTube)](https://www.youtube.com/watch?v=Vf7oJ6z2LCc)
- [Cleveland, W. S., & Devlin, S. J. (1988). "Locally Weighted Regression: An Approach to Regression Analysis by Local Fitting." Journal of the American Statistical Association, 83(403), 596–610.](https://www.jstor.org/stable/2289282)
- [Wikipedia: LOESS](https://en.wikipedia.org/wiki/Local_regression)
- [Statsmodels documentation: Nonparametric regression](https://www.statsmodels.org/stable/generated/statsmodels.nonparametric.smoothers_lowess.lowess.html)
- [Rob J Hyndman: STL Decomposition](https://otexts.com/fpppy/nbs/03-decomposition.html#sec-stl)


#### Advantages of STL
1. **Adaptability**:
    - STL can handle non-linear trends and varying seasonal patterns, making it suitable for complex time series data.

3. **Robustness**:
    - STL is robust to outliers and missing data, ensuring reliable decomposition in real-world scenarios.

 Note: STL assumes an additive model by default. While it can handle multiplicative models by transforming the data (e.g., log transformation), this requires additional preprocessing.


In [ ]:
stl = STL(
    data.select(pl.col(target_).forward_fill()),
    period=48,
)
res = stl.fit()
fig = plot_decomposition(
    time,
    res.observed.to_numpy().squeeze(),
    res.seasonal,
    res.trend,
    res.resid,
)
fig.update_xaxes(type="date", range=["2012-11-4", "2012-12-4"])
fig.show()

### STL Interpretation

- **Trend:** Still flat for this household.
- **Seasonality:** STL adapts to changes in the seasonal pattern over time.
- **Residuals:** Still some structure, suggesting more than one seasonal pattern.

Besides seasonality analysis, STL has several other practical applications in time series analysis:

1. **Forecasting**:  
    STL is commonly used as a preprocessing step for forecasting models, allowing separate modeling of trend and seasonal effects to improve forecast accuracy. We will have a look at how to use STL, or rather MSTL to forecast in later chapter
2. **Anomaly Detection**:  
    By examining the residual component after removing trend and seasonality, STL helps to identify unusual patterns or outliers in the data.
3. **Data Cleaning and Preprocessing**:  
    Removing trend and seasonal components with STL can produce a stationary residual series, which is often required for further statistical analysis or modeling.

## 6. MSTL: Multiple Seasonal-Trend Decomposition using LOESS

MSTL extends STL to handle multiple seasonalities (e.g., daily and weekly cycles). This is especially useful for electricity data, which often has both.


While STL is powerful for decomposing a time series into trend, seasonality, and residual components, it is limited to a **single seasonal period**. However, many real-world time series—such as electricity demand, web traffic, or retail sales—exhibit **multiple seasonal patterns** (e.g., daily and weekly cycles).

#### What is MSTL?

**MSTL** (Multiple Seasonal-Trend decomposition using LOESS) is an extension of STL that allows for the decomposition of time series with **multiple seasonalities**. MSTL iteratively applies STL to extract several seasonal components, each with its own period.

#### Mathematical Formulation

Given a time series $y_t$, MSTL decomposes it as:

$$
y_t = T_t + S^{(1)}_t + S^{(2)}_t + \cdots + S^{(K)}_t + R_t
$$

where:
- $T_t$ is the **trend** component,
- $S^{(k)}_t$ is the $k$-th **seasonal** component (e.g., daily, weekly),
- $R_t$ is the **remainder** (residual) component,
- $K$ is the number of seasonalities.

#### MSTL Algorithm (High-Level Steps)

1. **Initial Trend Estimation**:  
    Estimate the trend $T_t$ using LOESS smoothing.

2. **Iterative Seasonal Extraction**:  
    For each seasonal period $p_k$:
    - Remove the current estimate of the trend and all other seasonal components from $y_t$.
    - Apply STL to the detrended series to estimate $S^{(k)}_t$.

3. **Update Trend**:  
    After extracting all seasonal components, update the trend estimate using the seasonally adjusted series.

4. **Repeat**:  
    Iterate steps 2–3 until convergence.

5. **Compute Residual**:  
    $$
    R_t = y_t - T_t - \sum_{k=1}^K S^{(k)}_t
    $$

#### Example: Daily and Weekly Seasonality

For a time series with both daily ($p_1 = 48$ for half-hourly data) and weekly ($p_2 = 336$) seasonality:

$$
y_t = T_t + S^{(\text{daily})}_t + S^{(\text{weekly})}_t + R_t
$$

#### Why MSTL?

- **Captures Multiple Seasonal Patterns**:  
  MSTL can simultaneously model daily, weekly, and even yearly cycles.
- **Flexible and Robust**:  
  Like STL, MSTL uses LOESS smoothing, making it robust to outliers and adaptable to non-linear trends.
- **Improved Residuals**:  
  By removing multiple seasonalities, the residual component $R_t$ is closer to white noise, improving downstream modeling and forecasting.

#### References

- Hyndman, R.J., Wang, E., & Laptev, N. (2021). [Forecasting with Multiple Seasonal Patterns: Introducing MSTL](https://robjhyndman.com/papers/mstl.pdf)
- [Statsmodels MSTL Documentation](https://www.statsmodels.org/stable/generated/statsmodels.tsa.seasonal.MSTL.html)

---

In summary, MSTL generalizes STL to handle multiple seasonalities, making it a powerful tool for modern time series analysis where complex, overlapping cycles are common.

In [ ]:
stl = MSTL(
    data.select(pl.col(target_).forward_fill()),
    periods=[48, 7 * 48],  # daily and weekly
)
res = stl.fit()
fig = plot_decomposition(
    data.get_column(time_),
    res.observed,
    {"daily": res.seasonal[:, 0], "weekly": res.seasonal[:, 1]},
    res.trend,
    res.resid,
)
fig.update_xaxes(type="date", range=["2012-11-4", "2012-12-4"])
fig.show()

### MSTL Interpretation

- **Daily seasonality:** Strong, as expected.
- **Weekly seasonality:** Present but weaker.
- **Trend:** Still flat.
- **Residuals:** Now closer to white noise, meaning most structure is explained.

## 7. Quantifying Component Strength

We can measure how much of the variation is explained by trend or seasonality using variance ratios. This gives an objective measure of component importance.

### Measuring the Strength of Decomposed Components

The strength of each decomposed component (trend, seasonality, and residual) can be quantified based on their variation. This provides an objective measure of how much each component contributes to the overall time series, as opposed to relying solely on visual inspection.

#### Mathematical Formulation

1. **Trend Strength**:  
    The strength of the trend component is defined as:

    $$
    \text{Trend Strength} = \max\left(0, 1 - \frac{\text{Var}(R_t)}{\text{Var}(T_t + R_t)}\right)
    $$

    where:
    - $R_t$ is the residual component,
    - $T_t$ is the trend component,
    - $\text{Var}(\cdot)$ denotes the variance.

    This measures the proportion of variation explained by the trend relative to the combined trend and residual components.

2. **Seasonal Strength**:  
    For each seasonal component $S^{(k)}_t$, the strength is defined as:

    $$
    \text{Seasonal Strength}^{(k)} = 1 - \frac{\text{Var}(R_t)}{\text{Var}(S^{(k)}_t + R_t)}
    $$

    where:
    - $S^{(k)}_t$ is the $k$-th seasonal component,
    - $R_t$ is the residual component.

    This measures the proportion of variation explained by the seasonal component relative to the combined seasonal and residual components.

#### Advantages of Quantitative Measures

1. **Objectivity**:  
    Quantitative measures provide an objective way to assess the importance of each component, avoiding subjective biases that may arise from visual inspection of plots.

2. **Comparability**:  
    These measures allow for direct comparison of component strengths across different time series, enabling consistent evaluation.

3. **Automation**:  
    The strength measures can be computed programmatically, making them suitable for large-scale time series analysis where manual inspection is impractical.

4. **Insight into Model Fit**:  
    High residual variance relative to trend or seasonal components indicates that the decomposition may not fully capture the underlying structure of the time series, suggesting the need for model refinement.

In summary, measuring the strength of decomposed components based on their variation provides a robust and scalable approach to understanding the contributions of trend, seasonality, and residuals in a time series.

In [ ]:
def compute_strength(trend, seasonal, residual):
    residual_var = residual.var()
    trend_residual_var = (trend + residual).var()
    trend_strength = max(0, 1 - residual_var / trend_residual_var)
    if isinstance(seasonal, dict):
        for name, component in seasonal.items():
            seasonal_residual_var = (component + residual).var()
            strength = max(0, 1 - residual_var / seasonal_residual_var)
            print(f"Seasonal strength ({name}): {strength:.4f}")
    elif hasattr(seasonal, "ndim") and seasonal.ndim > 1:
        for i in range(seasonal.shape[1]):
            seasonal_residual_var = (seasonal[:, i] + residual).var()
            strength = max(0, 1 - residual_var / seasonal_residual_var)
            print(f"Seasonal strength (period {i + 1}): {strength:.4f}")
    else:
        seasonal_residual_var = (seasonal + residual).var()
        strength = max(0, 1 - residual_var / seasonal_residual_var)
        print(f"Seasonal strength: {strength:.4f}")
    print(f"Trend strength: {trend_strength:.4f}")
    return trend_strength

In [ ]:
_ = compute_strength(
    res.trend, {"daily": res.seasonal[:, 0], "weekly": res.seasonal[:, 1]}, res.resid
)

- The daily seasonal component explains a substantial portion of the variation (strength ≈ 0.58), indicating strong daily patterns in electricity usage.
- The weekly seasonal component is weaker (strength ≈ 0.22), suggesting some but less pronounced weekly cycles.
- The trend component is relatively weak (strength ≈ 0.20), confirming that there is little long-term change in the series.

## 8. Automatic Feature Extraction with `tsfeatures`

In addition to manual decomposition and strength calculations, you can automatically extract a variety of time series features—including STL-based features—using the [`tsfeatures`](https://github.com/Nixtla/tsfeatures) library. This library provides a convenient way to compute summary statistics and characteristics of time series, which are useful for exploratory analysis, feature engineering, and model selection.

#### STL Features via `tsfeatures`

The `tsfeatures` library includes the `stl_features` function, which computes several features based on STL decomposition, such as:

- **trend_strength**: Quantifies the strength of the trend component.
- **seasonal_strength**: Measures the strength of the seasonal component(s).
- **spikiness**: Indicates the presence of sharp spikes in the residuals.
- **linearity**: Measures the linearity of the trend.
- **curvature**: Measures the curvature of the trend.
- **seasonal_peak** and **seasonal_trough**: Identify the timing of seasonal peaks and troughs.

These features provide a quantitative summary of the time series structure and can be used for time series classification, clustering, or as input features for forecasting models.

#### Example Usage

You can compute STL features for your time series as follows:


In [ ]:
tsfeatures(
    data.select(pl.col(target_).forward_fill(), time_, id_).to_pandas(),
    freq=48,
    features=[stl_features],
)

## 9. Decomposition with Fourier Terms

Fourier decomposition is a powerful approach for modeling and extracting seasonal patterns in time series data, especially when the seasonality is complex or involves multiple overlapping cycles. Instead of estimating the seasonal component by averaging or smoothing, Fourier decomposition represents seasonality as a sum of sine and cosine functions with different frequencies.

#### How It Works

- **Fourier Series Representation:**  
    Any periodic function can be approximated by a sum of sine and cosine terms (Fourier series). For a time series with period $p$, the seasonal component $S_t$ can be modeled as:
    $$
    S_t = \sum_{k=1}^{K} \left[ a_k \sin\left(\frac{2\pi k t}{p}\right) + b_k \cos\left(\frac{2\pi k t}{p}\right) \right]
    $$
    where $K$ is the number of harmonics (pairs of sine and cosine terms), and $a_k$, $b_k$ are coefficients estimated from the data.

- **Multiple Seasonalities:**  
    By including Fourier terms for different periods (e.g., daily and weekly), the model can capture multiple seasonal patterns simultaneously.

- **Estimation:**  
    The coefficients of the Fourier terms are typically estimated using linear regression, often after removing the trend from the series (detrending). This allows for flexible and efficient modeling of seasonality.

#### Advantages

- **Flexibility:**  Fourier terms can approximate a wide range of seasonal patterns, including those that are not strictly constant over time.
- **Handles Multiple Seasonalities:**  Easily incorporates multiple seasonal cycles by adding terms for each relevant period.
- **Smoothness:**  The resulting seasonal component is smooth and continuous, avoiding abrupt changes.

#### Workflow

1. **Detrend the Series:** Remove the trend component (e.g., using LOESS or moving average).
2. **Generate Fourier Terms:** Create sine and cosine features for each seasonal period and harmonic.
3. **Fit Regression:** Regress the detrended series on the Fourier terms to estimate the seasonal component.
4. **Compute Residuals:** The remaining variation after removing trend and seasonality is the residual component.

Fourier decomposition is widely used in modern forecasting models (such as Prophet) and is especially effective for time series with complex or multiple seasonal patterns.

In [ ]:
y = data.select(pl.col(target_).forward_fill()).to_numpy().squeeze()
# 1. Extract trend using LOESS
loess_trend = lowess(y, np.arange(len(y)), frac=0.1, return_sorted=False)
# 2. Detrend the series
detrended_loess = y - loess_trend
# 3. Create Fourier terms for daily and weekly seasonality
features = [
    partial(fourier, season_length=48, k=10),
    partial(fourier, season_length=48 * 7, k=5),
]
data_fourier, _ = pipeline(
    data,
    features=features,
    freq="30m",
    h=1,
)
fourier_terms = data_fourier.select(pl.exclude([id_, time_, target_]))
# 4. Fit regression on Fourier terms to estimate seasonality
model = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("ridge_cv", RidgeCV(fit_intercept=False)),
    ]
)
model.fit(fourier_terms, detrended_loess)
seasonal_fourier = model.predict(fourier_terms)
# 5. Compute residuals
residual_fourier = detrended_loess - seasonal_fourier
# 6. Plot decomposition
plot_decomposition(
    time=time,
    observed=y,
    trend=loess_trend,
    seasonal=seasonal_fourier,
    resid=residual_fourier,
)

In [ ]:
_ = compute_strength(loess_trend, seasonal_fourier, residual_fourier)

- The strength output of the Fourier decomposition indicates that the seasonal component explains about 22% of the variation in the time series (seasonal strength ≈ 0.22), which suggests a weak but present seasonal pattern captured by the Fourier terms.
- The trend strength is very low (≈ 0.03), meaning the trend component explains only a small fraction of the variation, and most of the structure in the series is not attributable to a long-term trend.
- This aligns with the expectation for household electricity data, where seasonality may be present but the overall trend is minimal.